# [ParslFest 2025](https://parsl-project.org/parslfest/parslfest2025.html)

# [Accelerating QMCpy Notebook Tests with Parsl](https://www.figma.com/slides/k7EUosssNluMihkYTLuh1F/Parsl-Testbook-Speedup?node-id=1-37&t=WnKcu2QYO8JXvtpP-0)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QMCSoftware/QMCSoftware/blob/develop/demos/talk_paper_demos/parsel_fest_2025/parsl_fest_2025.ipynb)

Joshua Herman and Sou-Cheng Choi, QMCPy Developers

Aug 28 -- 29, 2025

Updated: Nov 30, 2025


**Requirements**:

* testbook : `pip install testbook==0.4.2`
* Parsl: `pip install parsl==2025.7.28`

In [1]:
try:
    import parsl as pl
except ModuleNotFoundError:
    !pip install -q parsl

In [2]:
import sys
import os
import time
import parsl as pl

# Ensure the path to the booktests directory is included (robust finder)
def _find_repo_root(start=os.getcwd()):
    cur = start
    while True:
        if os.path.exists(os.path.join(cur, 'pyproject.toml')):
            return cur
        parent = os.path.dirname(cur)
        if parent == cur:
            raise FileNotFoundError('repo root not found')
        cur = parent

sys.path.append(os.path.join(_find_repo_root(), 'test', 'booktests'))

# Configuration flags
force_compute = True
is_debug = False

# Create output directory if it doesn't exist
output_dir = "output"
os.makedirs(output_dir, exist_ok=True)

## 2. Parsl

1. Install and Configure Parsl
2. Run the tests in parallel with Parsl

### 2.1 Configure Parsl

In [3]:
from parsl.configs.htex_local import config

config.max_workers = 2
if not pl.config:
    pl.load(config)
    print("Parsl loaded with memory limits")
else:
    print("Parsl already configured")

Parsl already configured


### 2.2 Create a Parsl Test Runner

In [4]:
import parsl_test_runner
import inspect

# See only functions
print("Functions:")
functions = inspect.getmembers(parsl_test_runner, inspect.isfunction)
for name, func in functions:
    print(f"- {name}")
print("\n" + "="*50)

# Get help on specific function
print("Help for execute_parallel_tests:")
help(parsl_test_runner.execute_parallel_tests)

Functions:
- bash_app
- execute_parallel_tests
- generate_summary_report
- main
- reload_parsl_config

Help for execute_parallel_tests:
Help on function execute_parallel_tests in module parsl_test_runner:

execute_parallel_tests()
    Execute all testbook tests in parallel using Parsl



### 2.3 Run the Notebooks in Parallel with Parsl

In [5]:
par_fname = os.path.join(output_dir, f"parallel_times_{config.max_workers}.csv")
if (not os.path.exists(par_fname)) or force_compute:
    start_time2 = time.time()
    if is_debug:
        !(cd ../../.. && make booktests_parallel_no_docker TESTS="tb_quickstart tb_qmcpy_intro tb_quickstart tb_qmcpy_intro")
    else:
        !(cd ../../.. && make booktests_parallel_no_docker)
    parallel_time = time.time() - start_time2

    # read sequential time from file in output_dir
    seq_fname = os.path.join(output_dir, "sequential_time.csv")
    with open(seq_fname, "r") as f:
        sequential_time = float(f.read().strip())
        
    # compute speedup
    speedup = sequential_time / parallel_time
    print(f"Sequential time:  {sequential_time:.2f} seconds")
    print(f"Parallel time:    {parallel_time:.2f} seconds")
    print(f"Speedup by Parsl: {speedup:.2f}")

    # save parallel time to a file in output_dir
    with open(par_fname, "w") as f:
        f.write(f"workers,times,speedup\n")
        f.write(f"{config.max_workers},{parallel_time:.2f},{speedup:.2f}\n")

rm -fr demos/.ipynb_checkpoints/*checkpoint.ipynb && \
find demos -name '*.ipynb' | while read nb; do \
	base=$(basename "$nb" .ipynb); \
	test_base=$(echo "$base" | sed 's/[-.]/_/g'); \
	if [ "$base" != "parsl_fest_2025" ] && [ "$base" != "01_sequential" ] && [ "$base" != "02_parallel" ] && [ "$base" != "03_visualize_speedup" ] && ! ls test/booktests/tb_"$test_base".py > /dev/null 2>&1; then \
		echo "    Missing test for: $nb -> Expected: test/booktests/tb_$test_base.py"; \
	fi; \
done
Total notebooks:  35
Total test files: 32

Generating missing booktest files...
cd test/booktests/ && python generate_test.py --check-missing
No missing test files found.
rm -fr test/booktests/.ipynb_checkpoints/
chmod +x scripts/find_local_only_folders.sh > /dev/null 2>&1
for f in ; do \
	rm -f "$f"; > /dev/null 2>&1; \
done

Notebook tests with Parsl
pip install -q -e ".[test]"  && \
cd test/booktests/ && \
rm -fr *.eps *.jpg *.pdf *.png *.part *.txt *.log && rm -fr logs && rm -fr runinfo prob_failur

FileNotFoundError: [Errno 2] No such file or directory: 'output/sequential_time.csv'

In [ ]:
!date
!ls -ltr output